In [ ]:
! pip install pytubefix

In [ ]:
!pip install -q -U ffmpeg-python ffmpeg

: 

In [ ]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import os
def download_video(url):
  yt = YouTube(url, on_progress_callback = on_progress)
  ys = yt.streams.get_highest_resolution()
  file_path = ys.download()
  file_name = os.path.basename(file_path)
  return file_path, file_name

download_video("https://www.youtube.com/watch?v=E_Z3pc_hLJg")

('/content/An EVPI Equivalent for Markov Models: The Expected Value of Process Improvement.mp4',
 'An EVPI Equivalent for Markov Models: The Expected Value of Process Improvement.mp4')

In [ ]:
import cv2
import torch
import numpy as np
from transformers import CLIPProcessor, CLIPModel
from scipy.spatial.distance import cosine
import os
import ffmpeg

# Load the CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_frame_embedding(frame):
    # Convert the frame to the format required by the model
    inputs = processor(images=frame, return_tensors="pt")
    with torch.no_grad():
        # Extract the image embedding
        embedding = model.get_image_features(**inputs)
    return embedding.squeeze()

def detect_slide_changes(video_path, threshold=0.8, interval=1):
    cap = cv2.VideoCapture(video_path)
    success, frame = cap.read()

    if not success:
        print("Error: Failed to read the first frame.")
        return []

    prev_embedding = get_frame_embedding(frame)
    timestamp = 0
    timestamps = []

    while cap.isOpened():
        # Move to the next frame based on the interval
        cap.set(cv2.CAP_PROP_POS_MSEC, timestamp * 1000)
        ret, frame = cap.read()
        if not ret:
            break

        # Get the current frame's embedding
        curr_embedding = get_frame_embedding(frame)

        # Calculate the cosine similarity between consecutive embeddings
        similarity = 1 - cosine(prev_embedding, curr_embedding)

        if similarity < threshold:
            minutes = int(timestamp // 60)
            seconds = int(timestamp % 60)
            print(f"Slide changed at {minutes:02}:{seconds:02} minutes")  # Print slide change time
            timestamps.append(timestamp)

        # Update previous embedding and increment time
        prev_embedding = curr_embedding
        timestamp += interval

    cap.release()
    return timestamps

def split_video_into_chunks(video_path, timestamps, output_folder="video_chunks"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    timestamps.append(None)  # Add end marker for the last chunk
    for i, start_time in enumerate(timestamps[:-1]):
        end_time = timestamps[i + 1] if timestamps[i + 1] is not None else None
        output_path = os.path.join(output_folder, f"chunk_{i + 1}.mp4")

        # Convert start and end times to minutes and seconds for logging
        start_minutes = int(start_time // 60)
        start_seconds = int(start_time % 60)
        print(f"Chunk {i + 1} starts at {start_minutes:02}:{start_seconds:02} minutes")

        if end_time is None:
            ffmpeg.input(video_path, ss=start_time).output(output_path).run(overwrite_output=True)
        else:
            end_minutes = int(end_time // 60)
            end_seconds = int(end_time % 60)
            print(f"Chunk {i + 1} ends at {end_minutes:02}:{end_seconds:02} minutes")
            ffmpeg.input(video_path, ss=start_time, to=end_time).output(output_path).run(overwrite_output=True)

# Usage
video_path = "a.mp4"  # Replace with your video path
slide_change_timestamps = detect_slide_changes(video_path, threshold=0.8, interval=1)
split_video_into_chunks(video_path, slide_change_timestamps)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Slide changed at 00:33 minutes
Slide changed at 01:27 minutes
Slide changed at 02:20 minutes
Slide changed at 04:13 minutes
Slide changed at 06:09 minutes
Slide changed at 06:45 minutes
Chunk 1 starts at 00:33 minutes
Chunk 1 ends at 01:27 minutes
Chunk 2 starts at 01:27 minutes
Chunk 2 ends at 02:20 minutes
Chunk 3 starts at 02:20 minutes
Chunk 3 ends at 04:13 minutes
Chunk 4 starts at 04:13 minutes
Chunk 4 ends at 06:09 minutes
Chunk 5 starts at 06:09 minutes
Chunk 5 ends at 06:45 minutes
Chunk 6 starts at 06:45 minutes


In [ ]:
!pip install transformers
!pip install moviepy


In [ ]:
import torch
from transformers import BlipProcessor, BlipModel
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os

# Initialize BLIP model and processor
model = BlipModel.from_pretrained("Salesforce/blip-image-captioning-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

`BlipModel` is going to be deprecated in future release, please use `BlipForConditionalGeneration`, `BlipForQuestionAnswering` or `BlipForImageTextRetrieval` depending on your usecase.
Some weights of BlipModel were not initialized from the model checkpoint at Salesforce/blip-image-captioning-base and are newly initialized: ['logit_scale', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.word_embeddings.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'text_model.encoder.layer.0.attention.output.dense.bias', 'text_model.encoder.layer.0.attention.output.dense.weight', 'text_model.encoder.layer.0.attention.self.key.bias', 'text_model.encoder.layer.0.attention.self.key.weight', 'text_model.encoder.layer.0.attention.self.query.bias', 'text_model.encoder.layer.0.attention.self.query.weight', 'text_mo

In [ ]:
!pip install transformers
!pip install moviepy
!pip install torch
!pip install --upgrade transformers


In [ ]:

import torch
from transformers import CLIPProcessor, CLIPModel
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os

# Initialize CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")


In [ ]:

# Path to the input video
video_path = "a.mp4"
output_dir = "output_chunks2"
os.makedirs(output_dir, exist_ok=True)

# Load video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

# Parameters
embedding_diff_threshold = 0.5
scene_boundaries = [0]  # Start at frame 0
frame_skip = 5  # Process every 5th frame for speed-up

# Process frames with skipping
embedding_list = []
frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process every nth frame
    if frame_num % frame_skip == 0:
        # Prepare frame for CLIP
        inputs = processor(images=frame, return_tensors="pt").to(device)
        with torch.no_grad():
            embeddings = model.get_image_features(**inputs).squeeze()

        # Add current embedding to list
        embedding_list.append(embeddings)

        # Check for significant changes in embeddings
        if len(embedding_list) > 1:
            diff = torch.dist(embedding_list[-1], embedding_list[-2]).item()
            if diff > embedding_diff_threshold:
                current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                scene_boundaries.append(current_frame)

    frame_num += 1

cap.release()

# Extract video chunks based on scene boundaries
for i in range(len(scene_boundaries) - 1):
    start_time = scene_boundaries[i] / fps
    end_time = scene_boundaries[i + 1] / fps
    output_file = os.path.join(output_dir, f"chunk_{i + 1}.mp4")
    ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=output_file)
    print(f"Saved chunk {i + 1} from {start_time:.2f} to {end_time:.2f} as {output_file}")

print("Adaptive chunking completed.")


In [ ]:
!pip install transformers moviepy torch torchvision


In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import cv2
import os
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Initialize BLIP model and processor
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to('cuda')  # Move model to GPU
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# Path to the input video
video_path = "a.mp4"
output_dir = "output_chunks4"
os.makedirs(output_dir, exist_ok=True)

# Load video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps * 60)  # Number of frames in one minute

# Define threshold for transition detection
embedding_diff_threshold = 0.5
scene_boundaries = [0]  # Start at frame 0

# Process video frames in batches
embedding_list = []
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Only process one frame per minute
    if frame_count % frame_interval == 0:
        # Prepare frame for BLIP
        inputs = processor(images=frame, text=["Describe the image"], return_tensors="pt").to('cuda')  # Move inputs to GPU

        with torch.no_grad():
            outputs = model(**inputs)

            # Print the entire outputs to inspect its structure
            print(outputs)

            # Extract embeddings based on the actual structure
            if hasattr(outputs, 'image_embeds'):
                embeddings = outputs.image_embeds.squeeze()  # Extract image embeddings
            else:
                raise ValueError("Outputs do not contain expected attributes for embeddings.")

        # Append current embedding to list
        embedding_list.append(embeddings)

        # Check for significant differences in embeddings
        if len(embedding_list) > 1:
            diff = torch.dist(embedding_list[-1], embedding_list[-2]).item()
            if diff > embedding_diff_threshold:
                # Calculate frame number and add to boundaries
                current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                scene_boundaries.append(current_frame)

    frame_count += 1

cap.release()

# Extract video chunks based on scene boundaries
for i in range(len(scene_boundaries) - 1):
    start_time = scene_boundaries[i] / fps
    end_time = scene_boundaries[i + 1] / fps
    output_file = os.path.join(output_dir, f"chunk_{i + 1}.mp4")
    ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=output_file)
    print(f"Saved chunk {i + 1} from {start_time:.2f} to {end_time:.2f} as {output_file}")

print("Adaptive chunking completed.")


In [ ]:
# (pixel level )VideoDiffusion: A Generalized Video Generation Framework (2023): not good for our work
#Masked Video Modeling for Video Understanding (2023) : This approach focuses on predicting missing parts of video sequences, which can be useful for understanding context and filling in gaps in video information.


In [ ]:
!pip install torch torchvision transformers opencv-python


In [ ]:
# 1.VideoDiffusion
# Pros:
# It provides a generalized framework for video generation, which can be beneficial for synthesizing or interpolating frames in videos with PDF slides.
# Can capture temporal dependencies, which might help in understanding the flow of a lecture.
# Cons:
# It may be overly complex for simply identifying and analyzing slides.
# Requires significant computational resources and expertise to implement effectively.
# 2. TimeSformer
# Pros:
# Specifically designed for video classification, allowing it to capture both spatial and temporal information effectively.
# Could potentially classify segments of a lecture based on the content of slides.
# Cons:
# Depending on your specific use case, it might not be necessary to implement a full transformer model for simple slide detection or retrieval.
# Requires training and tuning, which can be resource-intensive.
# 3. VADER
# Pros:
# Focuses on video alignment and retrieval, making it suitable for scenarios where you want to compare and retrieve relevant parts of lecture videos.
# Useful for detecting differences between slides and aligning them for better retrieval.
# Cons:
# As with the other two, it may be more complex than necessary for simple slide analysis tasks

In [ ]:
import cv2
import os
import numpy as np

def detect_slide_changes(video_path, threshold=100000):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return []

    ret, prev_frame = cap.read()
    if not ret:
        print(f"Error: Could not read the first frame from {video_path}")
        return []

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    slide_changes = [0]
    frame_cache = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        #optical flow calculations can be computationally expensive
        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        motion_score = np.sum(magnitude)

        if motion_score > threshold:
            slide_changes.append(frame_count)
            prev_gray = gray
            frame_cache.append((frame_count, frame))
        elif frame_count % 60 == 0:  # Cache every 60th frame
            frame_cache.append((frame_count, frame))

        frame_count += 1

    cap.release()
    return slide_changes, frame_cache

def chunk_video(video_path, output_dir, slide_changes, frame_cache):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    chunk_count = 0

    for i in range(len(slide_changes) - 1):
        out_path = os.path.join(output_dir, f'chunk_{chunk_count}.avi')
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'XVID'), fps, frame_size)

        for j in range(slide_changes[i], slide_changes[i+1]):
            frame = next((f for fc, f in frame_cache if fc == j), None)
            if frame is not None:
                out.write(frame)

        out.release()
        chunk_count += 1

    cap.release()

video_path = 'a.mp4'
output_dir = 'video_chunks'
slide_changes, frame_cache = detect_slide_changes(video_path)
chunk_video(video_path, output_dir, slide_changes, frame_cache)


In [ ]:
# Frame Differencing

import cv2
import os

def detect_slide_changes(video_path, threshold=1000):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return []

    ret, prev_frame = cap.read()
    if not ret:
        print(f"Error: Could not read the first frame from {video_path}")
        return []

  #Instead of using dense optical flow, we can use frame differencing.
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    slide_changes = [0]
    frame_cache = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(prev_gray, gray)
        non_zero_count = cv2.countNonZero(diff)

        if non_zero_count > threshold:
            slide_changes.append(frame_count)
            prev_gray = gray
            frame_cache.append((frame_count, frame))
        elif frame_count % 10 == 0:  # Cache every 10th frame
            frame_cache.append((frame_count, frame))

        frame_count += 1

    cap.release()
    return slide_changes, frame_cache

def chunk_video(video_path, output_dir, slide_changes, frame_cache):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    chunk_count = 0

    for i in range(len(slide_changes) - 1):
        out_path = os.path.join(output_dir, f'chunk_{chunk_count}.avi')
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'XVID'), fps, frame_size)

        for j in range(slide_changes[i], slide_changes[i+1]):
            frame = next((f for fc, f in frame_cache if fc == j), None)
            if frame is not None:
                out.write(frame)

        out.release()
        chunk_count += 1

    cap.release()

video_path = 'a.mp4'
output_dir = 'video_chunks'
slide_changes, frame_cache = detect_slide_changes(video_path)
chunk_video(video_path, output_dir, slide_changes, frame_cache)


In [ ]:
!pip install opencv-python-headless pysaliency


In [9]:
# Saliency Detection

import cv2
import numpy as np
import os
import ffmpeg

def spectral_residual(frame):
    """Calculate the spectral residual of the given frame."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    f_transform = np.fft.fft2(gray_frame)
    f_transform_shifted = np.fft.fftshift(f_transform)

    magnitude = np.abs(f_transform_shifted)
    log_magnitude = np.log1p(magnitude)  # log(1 + x) to avoid log(0)
    spectral_residual = np.exp(log_magnitude - np.mean(log_magnitude))

    return spectral_residual

def detect_slide_changes(video_path, threshold=0.1):
    """Detect slide changes in a video and return frame indices for each chunk."""
    cap = cv2.VideoCapture(video_path)
    prev_spectral_residual = None
    timestamps = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        spectral_residual_frame = spectral_residual(frame)
        mean_residual = np.mean(spectral_residual_frame)

        if prev_spectral_residual is not None:
            if abs(mean_residual - np.mean(prev_spectral_residual)) > threshold:
                timestamps.append(frame_count / cap.get(cv2.CAP_PROP_FPS))  # Store time in seconds

        prev_spectral_residual = spectral_residual_frame
        frame_count += 1

    if frame_count > 0:
        timestamps.append(frame_count / cap.get(cv2.CAP_PROP_FPS))  # Add last timestamp

    cap.release()
    return timestamps

def split_video_into_chunks(video_path, timestamps, output_folder="video_chunks"):
    """Split the video into chunks based on the detected timestamps."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    timestamps.append(None)  # Add end marker for the last chunk
    for i, start_time in enumerate(timestamps[:-1]):
        end_time = timestamps[i + 1] if timestamps[i + 1] is not None else None
        output_path = os.path.join(output_folder, f"chunk_{i + 1}.mp4")

        # Convert start and end times to minutes and seconds for logging
        start_minutes = int(start_time // 60)
        start_seconds = int(start_time % 60)
        print(f"Chunk {i + 1} starts at {start_minutes:02}:{start_seconds:02} minutes")

        if end_time is None:
            # Use FFmpeg to extract the chunk starting from start_time to the end of the video
            ffmpeg.input(video_path, ss=start_time).output(output_path).run(overwrite_output=True)
        else:
            end_minutes = int(end_time // 60)
            end_seconds = int(end_time % 60)
            print(f"Chunk {i + 1} ends at {end_minutes:02}:{end_seconds:02} minutes")
            # Use FFmpeg to extract the chunk between start_time and end_time
            ffmpeg.input(video_path, ss=start_time, to=end_time).output(output_path).run(overwrite_output=True)

def main():
    video_path = 'a.mp4'  # Path to your video file
    timestamps = detect_slide_changes(video_path)
    split_video_into_chunks(video_path, timestamps)

if __name__ == "__main__":
    main()


Chunk 1 starts at 08:21 minutes


In [ ]:

#pixel level (SSIM)

import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import os
import ffmpeg

def detect_slide_changes(video_path, threshold=0.8, interval=1):
    cap = cv2.VideoCapture(video_path)
    
    # Attempt to read the first frame
    success, prev_frame = cap.read()
    if not success:
        print("Error: Failed to read the first frame.")
        return []

    # Convert the first frame to grayscale
    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    timestamp = 0

    # Store timestamps of slide changes
    timestamps = []
    
    while cap.isOpened():
        # Set the capture to the next frame based on interval
        cap.set(cv2.CAP_PROP_POS_MSEC, timestamp * 1000)
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculate SSIM between previous and current frame
        similarity = ssim(prev_frame, gray_frame)
        
        # If similarity is below the threshold, it's a slide change
        if similarity < threshold:
            # Store the timestamp
            minutes = int(timestamp // 60)
            seconds = int(timestamp % 60)
            print(f"Slide changed at {minutes:02}:{seconds:02} minutes")  # Print slide change time
            timestamps.append(timestamp)

        # Update the previous frame
        prev_frame = gray_frame
        timestamp += interval

    cap.release()
    return timestamps

def split_video_into_chunks(video_path, timestamps, output_folder="output_chunks"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    timestamps.append(None)  # End marker for the last chunk
    for i, start_time in enumerate(timestamps[:-1]):
        end_time = timestamps[i + 1] if timestamps[i + 1] is not None else None
        output_path = os.path.join(output_folder, f"chunk_{i + 1}.mp4")
        
        if end_time is None:
            ffmpeg.input(video_path, ss=start_time).output(output_path).run(overwrite_output=True)
        else:
            ffmpeg.input(video_path, ss=start_time, to=end_time).output(output_path).run(overwrite_output=True)

# Usage
video_path = "a.mp4"  # Replace with your video path
slide_change_timestamps = detect_slide_changes(video_path, threshold=0.8, interval=1)
split_video_into_chunks(video_path, slide_change_timestamps)


In [ ]:
# content-based scene detection approach -> less accurate , 3 or more sec faster


import cv2
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
import os

# Path to the input video
video_path = "/Users/emika/Downloads/a.mp4"
output_dir = "/Users/emika/Downloads/output_chunks"  # Directory to save the video chunks

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize VideoManager and SceneManager
video_manager = VideoManager([video_path])
scene_manager = SceneManager()
scene_manager.add_detector(ContentDetector(threshold=5.0))

# Start processing the video
video_manager.start()
scene_manager.detect_scenes(frame_source=video_manager)
scene_list = scene_manager.get_scene_list()

# Load the video with OpenCV
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second

print("Saving detected scenes as individual video files...")

# Loop through detected scenes and save each one
for i, scene in enumerate(scene_list):
    start_frame = scene[0].get_frames()
    end_frame = scene[1].get_frames()

    # Set up video writer for each scene
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    output_file = os.path.join(output_dir, f"scene_{i + 1}.mp4")
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Read frames and write to output file
    for frame_num in range(start_frame, end_frame + 1):
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    # Release the writer for the current scene
    out.release()
    print(f"Scene {i + 1} saved as {output_file}")

# Release resources
cap.release()
video_manager.release()
print("All scenes have been saved successfully.")
